<a href="https://colab.research.google.com/github/HatemMoushir/Shark-identification-1/blob/main/Shark%20identification_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install # ✅ تثبيت مكتبة بديلة
!pip install -q simple_image_download

# ✅ استيراد المكتبات
import os
from simple_image_download import simple_image_download as simp

# ✅ إعداد مجلد الصور
save_dir = "shark_fins"
os.makedirs(save_dir, exist_ok=True)

# ✅ قائمة الأنواع
shark_species = [
    ("Scalloped hammerhead", "Sphyrna lewini"),
    ("Great hammerhead", "Sphyrna mokarran"),
    ("Oceanic whitetip shark", "Carcharhinus longimanus"),
    ("Tiger shark", "Galeocerdo cuvier"),
    ("Silky shark", "Carcharhinus falciformis"),
    ("Blacktip shark", "Carcharhinus limbatus"),
    ("Dusky shark", "Carcharhinus obscurus"),
    ("Grey reef shark", "Carcharhinus amblyrhynchos"),
    ("Thresher shark", "Alopias vulpinus"),
    ("Shortfin mako", "Isurus oxyrinchus")
]

# ✅ تحميل الصور
response = simp.simple_image_download

for common, scientific in shark_species:
    folder = os.path.join(save_dir, common.replace(" ", "_"))

    # مجلدات الاسم الشائع والعلمي
    os.makedirs(os.path.join(folder, "common_name"), exist_ok=True)
    os.makedirs(os.path.join(folder, "scientific_name"), exist_ok=True)

    print(f"⬇️ تحميل الصور لـ {common}...")

    # تحميل صور الاسم الشائع
    response().download(f"{common} dorsal fin", 250, extensions=['.jpg', '.jpeg', '.png'])
    os.system(f"mv simple_images/{common.replace(' ', '_')}/* '{folder}/common_name/'")

    # تحميل صور الاسم العلمي
    response().download(f"{scientific} dorsal fin", 250, extensions=['.jpg', '.jpeg', '.png'])
    os.system(f"mv simple_images/{scientific.replace(' ', '_')}/* '{folder}/scientific_name/'")

print("✅ تم تحميل جميع الصور.")

from icrawler.builtin import GoogleImageCrawler
import os

# أنواع القروش المستهدفة مع كلمات البحث
shark_types = {
    "great_white": "great white shark dorsal fin",
    "tiger": "tiger shark dorsal fin",
    "bull": "bull shark dorsal fin"
}

# عدد الصور المطلوبة لكل نوع
num_images = 100

# المجلد الرئيسي لحفظ الصور
base_dir = "shark_fins_dataset"
os.makedirs(base_dir, exist_ok=True)

# تحميل الصور
for label, query in shark_types.items():
    save_dir = os.path.join(base_dir, label)
    os.makedirs(save_dir, exist_ok=True)

    print(f"\n🔎 Downloading up to {num_images} images for '{label}' ({query})...")

    crawler = GoogleImageCrawler(storage={'root_dir': save_dir})
    crawler.crawl(keyword=query, max_num=num_images, file_idx_offset=0)

    # عدّ الصور المحفوظة فعليًا
    downloaded = len([f for f in os.listdir(save_dir) if os.path.isfile(os.path.join(save_dir, f))])
    print(f"✅ Downloaded {downloaded} image(s) for '{label}'")

print("\n🎯 All downloads completed.")

In [ ]:
import shutil

shutil.make_archive("shark_dataset", 'zip', "shark_dataset")

'/content/shark_dataset.zip'

In [ ]:
from google.colab import files

files.download("shark_dataset.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from PIL import Image
import glob

def is_valid_image(path, min_size=(100, 100)):
    try:
        img = Image.open(path)
        img.verify()  # تأكد أن الملف صورة حقيقية
        img = Image.open(path)  # نحتاج نعيد فتحها بعد verify
        return img.size[0] >= min_size[0] and img.size[1] >= min_size[1]
    except:
        return False

# نحذف الصور غير الصالحة
def filter_invalid_images(root_dir):
    for class_dir in os.listdir(root_dir):
        class_path = os.path.join(root_dir, class_dir)
        for img_path in glob.glob(f"{class_path}/*"):
            if not is_valid_image(img_path):
                print(f"🗑️ Removing invalid: {img_path}")
                os.remove(img_path)

# تطبيق الفلترة
filter_invalid_images(dataset_dir)

🗑️ Removing invalid: ./shark_dataset/Blacktip_Shark/000024.jpg


In [ ]:

def resize_all_images(root_dir, size=(224, 224)):
    for class_dir in os.listdir(root_dir):
        class_path = os.path.join(root_dir, class_dir)
        for img_path in glob.glob(f"{class_path}/*"):
            try:
                img = Image.open(img_path)
                img = img.convert("RGB")  # تأكد من 3 قنوات
                img = img.resize(size)
                img.save(img_path)
            except Exception as e:
                print(f"❌ Failed resizing {img_path}: {e}")

# تطبيق التعديل
resize_all_images(dataset_dir)

In [ ]:
import os
import random
import shutil

def split_dataset(input_dir, output_dir, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
    random.seed(42)  # لتكرار نفس النتائج

    for class_name in os.listdir(input_dir):
        class_path = os.path.join(input_dir, class_name)
        if not os.path.isdir(class_path):
            continue

        images = os.listdir(class_path)
        random.shuffle(images)

        total = len(images)
        train_cut = int(train_ratio * total)
        val_cut = int((train_ratio + val_ratio) * total)

        splits = {
            "train": images[:train_cut],
            "val": images[train_cut:val_cut],
            "test": images[val_cut:]
        }

        for split_name, split_images in splits.items():
            split_class_dir = os.path.join(output_dir, split_name, class_name)
            os.makedirs(split_class_dir, exist_ok=True)

            for img in split_images:
                src = os.path.join(class_path, img)
                dst = os.path.join(split_class_dir, img)
                shutil.copy2(src, dst)

# تنفيذ التقسيم
split_dataset(dataset_dir, "/content/split_dataset")

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch

# نضبط حجم الصورة بما يتناسب مع النموذج (مثلاً 224x224)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [ ]:
# المسارات
train_path = "/content/split_dataset/train"
val_path = "/content/split_dataset/val"
test_path = "/content/split_dataset/test"

# تحميل البيانات
train_dataset = datasets.ImageFolder(train_path, transform=transform)
val_dataset = datasets.ImageFolder(val_path, transform=transform)
test_dataset = datasets.ImageFolder(test_path, transform=transform)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# طباعة عدد العينات
print(f"🧠 عدد صور التدريب: {len(train_dataset)}")
print(f"📊 عدد صور التحقق: {len(val_dataset)}")
print(f"🔍 عدد صور الاختبار: {len(test_dataset)}")

# عرض الفئات
print(f"📦 الفئات: {train_dataset.classes}")

🧠 عدد صور التدريب: 1134
📊 عدد صور التحقق: 244
🔍 عدد صور الاختبار: 246
📦 الفئات: ['Blacktip_Shark', 'Great_White_Shark', 'Hammerhead_Shark', 'Silky_Shark', 'Tiger_Shark', 'Whale_Shark', 'Whitetip_Shark']


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)

        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)

        self.fc1 = nn.Linear(64 * 28 * 28, 256)
        self.fc2 = nn.Linear(256, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # [B, 16, 112, 112]
        x = self.pool(F.relu(self.conv2(x)))  # [B, 32, 56, 56]
        x = self.pool(F.relu(self.conv3(x)))  # [B, 64, 28, 28]
        x = x.view(-1, 64 * 28 * 28)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

In [ ]:

# تحديد عدد الفئات من بيانات التدريب
num_classes = len(train_dataset.classes)

# تحميل على CUDA أو CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# إنشاء النموذج
model = CNNModel(num_classes).to(device)

print(model)

CNNModel(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=50176, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=7, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


In [ ]:
import torch.optim as optim

# دالة الفقد: تصنيف متعدد الفئات => CrossEntropyLoss
criterion = nn.CrossEntropyLoss()

# المُحسّن: Adam مع learning rate
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs = 5

for epoch in range(num_epochs):
    running_loss = 0.0
    model.train()

    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # تصفير الـ gradients
        optimizer.zero_grad()

        # التنبؤ
        outputs = model(inputs)

        # حساب الفقد
        loss = criterion(outputs, labels)

        # التدرج العكسي والتحديث
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {running_loss/len(train_loader):.4f}")

print("✅ التدريب خلص.")

Epoch [1/5] - Loss: 1.9652
Epoch [2/5] - Loss: 1.8655
Epoch [3/5] - Loss: 1.8048
Epoch [4/5] - Loss: 1.7774
Epoch [5/5] - Loss: 1.7443
✅ التدريب خلص.


In [ ]:
def evaluate_model(model, val_loader):
    model.eval()  # نمط التقييم
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'✅ Accuracy on validation set: {accuracy:.2f}%')

# نفذ التقييم
evaluate_model(model, val_loader)

✅ Accuracy on validation set: 25.00%


In [ ]:


!pip install torch torchvision

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

# ✅ استخدام GPU لو متاح
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ تحميل البيانات
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

train_dataset = datasets.ImageFolder("shark_data_split/train", transform=transform)
test_dataset = datasets.ImageFolder("shark_data_split/test", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# ✅ تحميل نموذج ResNet18
model = models.resnet18(pretrained=True)
num_classes = len(train_dataset.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

# ✅ إعدادات التدريب
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# ✅ تدريب سريع (5 epochs فقط كبداية)
epochs = 5
for epoch in range(epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    acc = correct / total * 100
    print(f"📦 Epoch [{epoch+1}/{epochs}] - Loss: {total_loss:.4f} - Accuracy: {acc:.2f}%")

# ✅ اختبار سريع
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

test_acc = correct / total * 100
print(f"\n🧪 Test Accuracy: {test_acc:.2f}%")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 134MB/s]


📦 Epoch [1/5] - Loss: 9.0012 - Accuracy: 48.55%
📦 Epoch [2/5] - Loss: 3.1816 - Accuracy: 86.96%
📦 Epoch [3/5] - Loss: 2.1911 - Accuracy: 93.48%
📦 Epoch [4/5] - Loss: 1.5495 - Accuracy: 92.75%
📦 Epoch [5/5] - Loss: 1.3707 - Accuracy: 92.75%


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(



🧪 Test Accuracy: 59.46%


In [ ]:
# ✅ تثبيت المكتبة
!pip install icrawler

# ✅ استيراد الأدوات
import os
from icrawler.builtin import BingImageCrawler  # ممكن نبدّل GoogleImageCrawler لو أردت

# ✅ قائمة بأسماء القروش الشائعة
shark_types = {
    "Blacktip_Shark": "Blacktip shark fin above water",
    "Whitetip_Shark": "Oceanic whitetip shark dorsal fin above water",
    "Hammerhead_Shark": "Hammerhead shark fin above water",
    "Whale_Shark": "Whale shark fin above water",
    "Silky_Shark": "Silky shark fin above water",
    "Tiger_Shark": "Tiger shark fin above water",
    "Great_White_Shark": "Great white shark fin above water"
}

# ✅ إنشاء مجلد البيانات
dataset_dir = "./shark_dataset"
os.makedirs(dataset_dir, exist_ok=True)

# ✅ تحميل الصور
for folder_name, query in shark_types.items():
    save_dir = os.path.join(dataset_dir, folder_name)
    os.makedirs(save_dir, exist_ok=True)

    print(f"\n🔽 Now downloading: {folder_name} ...")
    crawler = BingImageCrawler(storage={"root_dir": save_dir})
    crawler.crawl(keyword=query, max_num=500)

In [ ]:
from PIL import Image
import os

def clean_corrupt_images(dataset_path):
    removed = 0
    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                with Image.open(file_path) as img:
                    img.verify()  # تحقق فقط دون فتح كامل
            except (IOError, SyntaxError):
                print(f"❌ Removing corrupt image: {file_path}")
                os.remove(file_path)
                removed += 1
    print(f"\n✅ Cleaning done. Removed {removed} corrupt images.")

clean_corrupt_images("shark_fins_dataset")

❌ Removing corrupt image: shark_fins_dataset/tiger/000005.jpg
❌ Removing corrupt image: shark_fins_dataset/tiger/000012.jpg
❌ Removing corrupt image: shark_fins_dataset/great_white/000030.jpg
❌ Removing corrupt image: shark_fins_dataset/great_white/000053.jpg

✅ Cleaning done. Removed 4 corrupt images.


In [ ]:

import os
import shutil
import random

def split_dataset(source_dir, train_ratio=0.8):
    random.seed(42)
    target_base = "shark_data_split"
    for class_name in os.listdir(source_dir):
        class_path = os.path.join(source_dir, class_name)
        if not os.path.isdir(class_path):
            continue

        images = os.listdir(class_path)
        random.shuffle(images)
        split_idx = int(len(images) * train_ratio)

        train_dir = os.path.join(target_base, "train", class_name)
        test_dir = os.path.join(target_base, "test", class_name)
        os.makedirs(train_dir, exist_ok=True)
        os.makedirs(test_dir, exist_ok=True)

        for i, img in enumerate(images):
            src_path = os.path.join(class_path, img)
            dst_dir = train_dir if i < split_idx else test_dir
            shutil.copy2(src_path, os.path.join(dst_dir, img))

    print("\n✅ Dataset split complete!")

split_dataset("shark_fins_dataset")


✅ Dataset split complete!


In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# التحويلات الأساسية للصور
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# تحميل البيانات
train_data = datasets.ImageFolder("shark_data_split/train", transform=transform)
test_data = datasets.ImageFolder("shark_data_split/test", transform=transform)

train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
test_loader = DataLoader(test_data, batch_size=16, shuffle=False)

print(f"✅ Loaded {len(train_data)} training images and {len(test_data)} testing images.")
print(f"Classes: {train_data.classes}")

✅ Loaded 138 training images and 37 testing images.
Classes: ['bull', 'great_white', 'tiger']


In [ ]:
import os

for root, dirs, files in os.walk("shark_data_split"):
    level = root.replace("shark_data_split", "").count(os.sep)
    indent = " " * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    for f in files[:3]:  # عرض أول 3 صور فقط من كل مجلد
        print(f"{indent}  {f}")

shark_data_split/
  train/
    tiger/
      000051.jpg
      000056.jpg
      000052.jpg
    great_white/
      000009.jpg
      000051.jpg
      000056.jpg
    bull/
      000009.jpg
      000051.jpg
      000048.jpg
  test/
    tiger/
      000048.jpg
      000035.jpg
      000014.jpg
    great_white/
      000048.png
      000052.jpg
      000004.jpg
    bull/
      000014.png
      000037.jpg
      000038.jpg


In [ ]:
import os
import shutil
import random

# Define the base directory for the downloaded images and the new split dataset directory
base_download_dir = "shark_fins_dataset"
base_split_dir = "shark_data_split"

# Define the train and test split ratio
train_ratio = 0.8
test_ratio = 1.0 - train_ratio

# Create the base split directory
os.makedirs(base_split_dir, exist_ok=True)

# Iterate through the downloaded shark types (folders)
for shark_type in os.listdir(base_download_dir):
    shark_type_dir = os.path.join(base_download_dir, shark_type)

    # Ensure it's a directory
    if os.path.isdir(shark_type_dir):
        # Create corresponding train and test directories in the split dataset directory
        train_dir = os.path.join(base_split_dir, "train", shark_type)
        test_dir = os.path.join(base_split_dir, "test", shark_type)
        os.makedirs(train_dir, exist_ok=True)
        os.makedirs(test_dir, exist_ok=True)

        # Get the list of image files and shuffle it
        images = [f for f in os.listdir(shark_type_dir) if os.path.isfile(os.path.join(shark_type_dir, f))]
        random.shuffle(images)

        # Calculate the number of images for training and testing
        num_train = int(len(images) * train_ratio)
        # num_test = len(images) - num_train # No need to calculate num_test explicitly

        # Split the images and copy them to the new directories
        train_images = images[:num_train]
        test_images = images[num_train:]

        print(f"Splitting {shark_type}: {len(train_images)} for training, {len(test_images)} for testing")

        for img in train_images:
            src = os.path.join(shark_type_dir, img)
            dst = os.path.join(train_dir, img)
            shutil.copyfile(src, dst)

        for img in test_images:
            src = os.path.join(shark_type_dir, img)
            dst = os.path.join(test_dir, img)
            shutil.copyfile(src, dst)

print("\nData split completed.")